In [29]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [30]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [31]:
from songs import *
import copy
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(copy.deepcopy(song))
    while song.transpose(-1):
        Songs.append(copy.deepcopy(song))
Songs = []
for i in range(1, 35):
    addAllTransposedVersions(Songs, Song('basic midi/track (' + str(i) + ').mid'))
    
def NextBatch():
    batch = np.empty((64, 13*128))
    y = []
    for i in range(64):
        song = copy.deepcopy(Songs[np.random.randint(0, len(Songs))])
        y.append([np.random.randint(0, 9) * np.random.randint(0, 9)])
        song.noise(y[-1][0])
        batch[i] = song.notes.flatten()
    return batch, (1 - np.array(y) / 64)

ERROR! out of range!
ERROR! out of range!
ERROR! out of range!


In [59]:
all_songs = np.array([song.notes for song in Songs])

In [60]:
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.001
training_steps = 20000
batch_size = all_songs.shape[0]
display_step = 200

# Network Parameters
num_input = 13
timesteps = 128 # timesteps
num_hidden = 100 # hidden layer num of features

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])

# Define weights
w = tf.Variable(tf.random_normal([num_hidden, num_input]))
b = tf.Variable(tf.random_normal([num_input]))

class lstm:
    def __init__(self, name, x, num_of_neurons):
        with tf.variable_scope(name):
            self.lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
            self.hidden_state = tf.zeros([batch_size, num_of_neurons])
            self.current_state = tf.zeros([batch_size, num_of_neurons])
            state = self.hidden_state, self.current_state
            self.outputs = []
            for inp in x:
                output, state = self.lstm_cell(inp, state)
                self.outputs.append(output)

_X = tf.unstack(X, timesteps, 1)
first_layer = lstm("fl", _X, num_hidden)
second_layer = lstm("sl", first_layer.outputs, num_hidden)

logits = tf.convert_to_tensor([tf.sigmoid(tf.matmul(second_layer.outputs[i], w) + b) for i in range(timesteps)])

# Define loss and optimizer
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=X))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
loss_op = -tf.reduce_mean(0.97 * tf.multiply(tf.convert_to_tensor(_X), tf.log(logits)) +
                         0.03 * tf.multiply(1. - tf.convert_to_tensor(_X), tf.log(1. - logits)))
#train_op = optimizer.minimize(loss_op)
train_op = tf.contrib.layers.optimize_loss(
    loss_op, tf.contrib.framework.get_global_step(), optimizer=optimizer,
    learning_rate=learning_rate)

# Evaluate model (with test logits, for dropout to be disabled)
#correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [61]:
# Start training
sess = tf.Session()
# Run the initializer
sess.run(init)

for step in range(0, training_steps):
    batch_x = all_songs#mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    
    # Run optimization op (backprop)
    sess.run(train_op, feed_dict={X: batch_x})
    if step % display_step == 0:
        # Calculate batch loss and accuracy
        L = sess.run(loss_op, feed_dict={X: batch_x})
        print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(L))

print("Optimization Finished!")

Step 0, Minibatch Loss= 0.0458
Step 200, Minibatch Loss= 0.0453
Step 400, Minibatch Loss= 0.0449
Step 600, Minibatch Loss= 0.0445
Step 800, Minibatch Loss= 0.0442
Step 1000, Minibatch Loss= 0.0438
Step 1200, Minibatch Loss= 0.0435
Step 1400, Minibatch Loss= 0.0432
Step 1600, Minibatch Loss= 0.0429
Step 1800, Minibatch Loss= 0.0427
Step 2000, Minibatch Loss= 0.0424
Step 2200, Minibatch Loss= 0.0422
Step 2400, Minibatch Loss= 0.0420
Step 2600, Minibatch Loss= 0.0418
Step 2800, Minibatch Loss= 0.0416
Step 3000, Minibatch Loss= 0.0415
Step 3200, Minibatch Loss= 0.0413
Step 3400, Minibatch Loss= 0.0412
Step 3600, Minibatch Loss= 0.0410
Step 3800, Minibatch Loss= 0.0409
Step 4000, Minibatch Loss= 0.0408
Step 4200, Minibatch Loss= 0.0407
Step 4400, Minibatch Loss= 0.0405
Step 4600, Minibatch Loss= 0.0404
Step 4800, Minibatch Loss= 0.0403
Step 5000, Minibatch Loss= 0.0403
Step 5200, Minibatch Loss= 0.0402
Step 5400, Minibatch Loss= 0.0401
Step 5600, Minibatch Loss= 0.0400
Step 5800, Minibatch 

In [111]:
all_songs = np.array([song.notes for song in Songs])
batch_x = all_songs
batch_x = batch_x.reshape((69, timesteps, num_input))

In [112]:
outp = np.array(sess.run(logits, feed_dict={X: batch_x}))

In [113]:
first_song = outp[:, 0, :]

In [114]:
first_song[all_songs[0][:32] == 1]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 128 but corresponding boolean dimension is 32
  """Entry point for launching an IPython kernel.


array([ 0.35305798,  0.40715349,  0.43449512,  0.51579624,  0.4916901 ,
        0.57061625,  0.56180316,  0.54283768,  0.56193489,  0.53043222,
        0.55983573,  0.52543014,  0.53507745,  0.51688242], dtype=float32)

In [115]:
first_song[all_songs[0][32:64] == 1]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 128 but corresponding boolean dimension is 32
  """Entry point for launching an IPython kernel.


array([ 0.35305798,  0.40715349,  0.43449512,  0.51579624,  0.4916901 ,
        0.57061625,  0.56180316,  0.54283768,  0.56193489,  0.53043222,
        0.55983573,  0.52543014,  0.53507745], dtype=float32)

Проблема: LSTM ни фига не запоминает

In [116]:
new_songs = [MySong() for i in range(69)]

In [117]:
for i in range(128):
    outp = np.array(sess.run(logits, feed_dict={X: batch_x}))
    for s in range(69):
        next_one = outp[i, s, :]
            
        next_one[next_one < 0.65] = 0
        next_one[next_one != 0] = 1
        
        if i >= 8:
            batch_x[s, i] = next_one
            new_songs[s].add(next_one)
        else:
            new_songs[s].add(batch_x[s, i])

In [118]:
new_songs[0].play()